In [ ]:
import pandas as pd
import time
from openai import OpenAI
from tqdm import tqdm

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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",  
)

full_df = pd.read_csv("shuffled_dataset.csv")

try:
    checkpoint_df = pd.read_csv("checkpoint_main_idea.csv")
    start_idx = len(checkpoint_df)
    print(f"Resuming from row {start_idx}")
except FileNotFoundError:
    checkpoint_df = full_df.copy()
    checkpoint_df["main_idea"] = ""
    start_idx = 0
    print("Starting from scratch")

for idx in tqdm(range(start_idx, len(full_df)), desc="Generating Main Ideas"):
    row = full_df.iloc[idx]
    prompt = f"""Given the YouTube title and transcript below, analyze the content and generate a single, detailed "Main Idea" in the voice of a YouTube creator asking an AI to make a video.

The "Main Idea" should:

Be slightly longer and more descriptive than the title

Clearly reflect what the video is about based on the transcript

Sound like a natural, casual request from a creator (e.g., "Make a video that...")

Vary in structure and focus across examples — don't repeat the same phrasing

Be specific and imaginative, giving a clear sense of what the user wants the video to achieve

Write this in a single sentence that captures the viewer’s or creator's intent.

Title: {row['title']}
Transcript: {row['transcript']}

Main Idea:"""

    try:
        completion = client.chat.completions.create(
            model="qwen/qwen2.5-vl-3b-instruct:free", 
            messages=[
                {"role": "user", "content": prompt}
            ],
            max_tokens=60,
        )
        if completion and completion.choices and completion.choices[0].message:
            main_idea = completion.choices[0].messages.content.strip()
        else:
            raise ValueError("Empty response from API")

    except Exception as e:
        print(f"\nError at index {idx}: {e}")
        print("Stopping execution. You can resume after fixing the issue or changing the API key.")
        break  

  
    checkpoint_df.at[idx, "main_idea"] = main_idea

    
    if idx % 5 == 0:
        checkpoint_df.to_csv("checkpoint_main_idea.csv", index=False)


checkpoint_df.to_csv("checkpoint_main_idea.csv", index=False)
print("Progress saved.")


Resuming from row 349


Generating Main Ideas: 0it [00:00, ?it/s]

Progress saved.


In [4]:
new_df = pd.read_csv('checkpoint_main_idea.csv')

In [9]:
new_df['main_idea'].isnull().sum()

349

In [6]:
new_df['main_idea'][43]

nan

In [7]:
new_df

,niche,video_link,title,channel_title,published_at,duration,duration_seconds,thumbnail_url,description,tags,transcript,region,video_id,view_count,like_count,comment_count,main_idea
0,fighter jets,https://www.youtube.com/watch?v=yeD25bjiA-E,Russian Air Force Receives First Su-35S 4++ Fi...,Military Gear,2025-04-16T12:01:49Z,PT8M11S,491.0,https://i.ytimg.com/vi/yeD25bjiA-E/hqdefault.jpg,Russia is soaring into a new era of air domina...,"su35s, su 35s, su-35s, russia's su-35s, russia...",how far has Russia gone in strengthening its a...,US,NaN,NaN,NaN,NaN,NaN
1,tech news,https://www.youtube.com/watch?v=x2WtHZciC74,Claude 3.7 goes hard for programmers…,Fireship,2025-02-25T18:27:50Z,PT5M49S,349.0,https://i.ytimg.com/vi/x2WtHZciC74/hqdefault.jpg,"Try Convex for free, the only database designe...","webdev, app development, lesson, tutorial",yesterday anthropic finally released Claude 3....,US,NaN,NaN,NaN,NaN,NaN
2,NaN,https://www.youtube.com/watch?v=EmJpb9eLplI,4 TYPES OF BODYFAT YOU MUST KNOW (The Fat Scie...,ABHINAV MAHAJAN,2018-04-23T16:05:12Z,PT8M33S,513.0,https://i.ytimg.com/vi/EmJpb9eLplI/hqdefault.jpg,Buy Genuine Supplements from MYPROTEIN : https...,"types of body fat, how many types of fat, fat ...",NaN,US,NaN,NaN,NaN,NaN,NaN
3,MMA,https://www.youtube.com/watch?v=THrr1VDYDdo,Rodtang Jitmuangnon (Thailand) vs Edgar Tabare...,That's why MMA!,2025-02-15T10:43:49Z,PT9M17S,557.0,https://i.ytimg.com/vi/THrr1VDYDdo/hqdefault.jpg,Professional Thai flyweight Muay Thai fighter ...,"emelianenko, bellator, MMA, UFC, MMA Highlight...",David versus goath no doubt Rod Tang 25 tbar 2...,US,NaN,NaN,NaN,NaN,NaN
4,fighter jets,https://www.youtube.com/watch?v=MjtQC4o6Ux8,How Iran Built the Saegheh Fighter Jet Under H...,SHKamran,2025-04-15T09:14:46Z,PT8M14S,494.0,https://i.ytimg.com/vi/MjtQC4o6Ux8/hqdefault.jpg,"Despite international sanctions, Iran develope...","Iran fighter jet, Saegheh aircraft, Iranian Ai...",how Iran built the SEI fighter jet under harsh...,US,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,"personal finance , investing",https://www.youtube.com/watch?v=EAhmU5i4YmU,Bajaj Housing Finance Unlocks 64% Shares What...,Equentis,2025-04-15T12:57:58Z,PT7M50S,470.0,https://i.ytimg.com/vi/EAhmU5i4YmU/hqdefault.jpg,"In today’s video, we’ll discuss the latest sto...","bajaj housing finance, bajaj housing finance s...",NaN,US,NaN,NaN,NaN,NaN,NaN
345,Economy,https://www.youtube.com/watch?v=3G-vTV9SVW4,What A U.S. Economy Under Trump Will Look Like,CNBC,2024-11-09T17:00:02Z,PT6M8S,368.0,https://i.ytimg.com/vi/3G-vTV9SVW4/hqdefault.jpg,"President-elect Donald Trump, is set to inheri...","economy, us news, federal reserve, economics, ...","The President-elect of the\nUnited States, Don...",US,NaN,NaN,NaN,NaN,NaN
346,Economy,https://www.youtube.com/watch?v=1l-jN8V_puk,Mass government layoffs: Impact on the labor f...,CNBC Television,2025-02-20T14:11:10Z,PT8M11S,491.0,https://i.ytimg.com/vi/1l-jN8V_puk/hqdefault.jpg,"Diana Furchtgott-Roth, former U.S. Department ...","Squawk Box U.S., CNBC, business news, finance ...",">> Joe okay. >> Steve, stay with us. Coming of...",US,NaN,NaN,NaN,NaN,NaN
347,NaN,https://www.youtube.com/watch?v=1UvuBYUbFk0,Digestive System,Amoeba Sisters,2021-02-01T02:30:00Z,PT8M43S,523.0,https://i.ytimg.com/vi/1UvuBYUbFk0/hqdefault.jpg,Join the Amoeba Sisters for a brief tour throu...,"digestion, how digestion works, digestive syst...",It is no secret that I love the platypus.\nMan...,US,NaN,NaN,NaN,NaN,NaN
